In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import gaussian_kde

from skfda.datasets import make_gaussian_process
from skfda.misc.covariances import *
from skfda.misc.metrics import l2_distance, l2_norm
from skfda.preprocessing.dim_reduction.feature_extraction import FPCA
from skfda.representation.basis import BSpline, Fourier

In [ ]:
time_range = (0,25)

def get_fpca(n_components=1, style=None):
    if style is None:
        fpca = FPCA(n_components=n_components)
        basis = None
    else:
        basis = get_basis(style, n_components)
        fpca = FPCA(n_components=n_components, components_basis=basis)
    return fpca, basis 

def get_basis(style, n_components):
    if style == 'bspline':
        basis = BSpline(n_basis=n_components)
    elif style == 'fourier':
        basis = Fourier(domain_range=time_range, n_basis=n_components, period=None)
    return basis

# decide on a basis for the FPCA
style = 'bspline'

Idea: create three covariance functions and sample Gaussian processes. Then do FPCA and compute Functional Graphical Lasso. Let the variables be called $v_0$, $v_1$ and $v_2$. 

In [ ]:
n_var = 3
n_ts = 100
n_samples = 10**3

# define three variables
cov1 = Gaussian(variance=0.5, length_scale=8.0)
cov2 = Gaussian(variance=0.5, length_scale=4.0)
cov3 = Exponential(variance=0.5, length_scale=8.0)
#cov3 = Brownian(variance=0.1)

all_cov = [cov1, cov2, cov3]

In [ ]:
# plot covariance functions
_ = cov1.heatmap()
_ = cov2.heatmap()
_ = cov3.heatmap()

In [ ]:
# sample gaussian processes
samples = list()
labels = list()

for j in np.arange(n_var):

    _ds = make_gaussian_process(
            n_samples=n_samples,
            n_features=n_ts,
            start=time_range[0],
            stop=time_range[1],
            cov=all_cov[j],
            random_state=20
            )
    
    
    _lb = np.array([f'v{j}'] * n_samples)
    
    samples.append(_ds)
    labels.append(_lb)


In [ ]:
# plot sampled time series

colors = ['darkred', 'C1', 'grey']

fig, axs = plt.subplots(n_var,1, figsize=(10,12))

for j in range(n_var):
    ax = axs.ravel()[j]
    samples[j].plot(axes=ax, group=labels[j], group_colors={f'v{j}': colors[j]}, alpha=0.2, lw=0.95)
    ax.set_title(f'Sampled time series for v{j}')

In [ ]:
# do FPCA, reconstruct and plot deviation
q = 7
fig, axs = plt.subplots(n_var, 2, figsize=(20,12))

for j in range(n_var):
    if style is None:
        this_ds = samples[j]
    else:
        basis = get_basis(style, q)
        this_ds = samples[j].to_basis(basis)
        
    # do FPCA
    fpca,_ = get_fpca(n_components=q, style=style)
    fpca.fit(this_ds)
    
    # reconstruct ds from FPCA
    recov_ds = fpca.inverse_transform(fpca.transform(this_ds))
    
    # reconvert to grid if necessary
    if style is None:
        diff = this_ds - recov_ds
    else:
        diff = samples[j] - recov_ds.to_grid(samples[j].grid_points)
        
    ax = axs[j,0]
    recov_ds.plot(axes=ax, group=labels[j], group_colors={f'v{j}': colors[j]}, alpha=0.3, lw=0.95)
    ax.set_title(f'FPCA transformed time series for v{j}')
    
    ax2 = axs[j,1]
    diff.plot(axes=ax2, group=labels[j], group_colors={f'v{j}': colors[j]}, alpha=0.3, lw=0.95)
    ax2.set_title(f'Recovery error of FPCA for v{j}')
    ax2.set_ylim(ax.get_ylim())
    
    

In [ ]:
fpca.components_basis

## Fourth variable construction

We want to create a fourth variable that is constructed using the first variable $v_1$ as follows: we do FPCA, then permute the coefficients and then retransform to a time series, but with the basis not permuted. This creates a variable with correlated singular values (wrt FPCA), but where the relation is less obvious if only looking at the time series.

In [ ]:
# from now on use 9 components
n_comp = 9

In [ ]:
# compute FPCA coefficients
if style is None:
    v0_ds = samples[0]
    fpca,_ = get_fpca(n_components=n_comp, style=style)
    fpca.fit(v0_ds)
    
else:
    basis = get_basis(style, n_comp)
    v0_ds = samples[0].to_basis(basis)        
    # do FPCA
    fpca,_ = get_fpca(n_components=n_comp, style=style)
    fpca.fit(v0_ds)

# permute
Z = fpca.transform(v0_ds)    
print(Z.shape)   
rng = np.random.default_rng(1917)
_perm = rng.permutation(n_comp)
Z = Z[:,_perm]

# retransform to time series
# (see: https://github.com/GAA-UAM/scikit-fda/blob/ab9da5c17522ad1b4ef7493bd908d0f1de6f2aec/skfda/preprocessing/dim_reduction/_fpca.py#L543)
if style is None:
    W = fpca.components_.data_matrix
    additional_args = {"data_matrix": np.einsum('nc,c...->n...', Z, W,),}
    v3_ds = fpca.mean_.copy(**additional_args, sample_names=(None,) * len(Z),) + fpca.mean_
else:
    W = fpca.components_.coefficients
    additional_args = {"coefficients": Z @ W,}
    v3_ds = fpca.mean_.copy(**additional_args, sample_names=(None,) * len(Z),) + fpca.mean_
    v3_ds = v3_ds.to_grid(samples[0].grid_points)



In [ ]:
n_var += 1

# add new time series
samples.append(v3_ds)
_lb3 = np.array([f'v3'] * n_samples)
labels.append(_lb3)
colors.append('steelblue')

# plot again
fig, axs = plt.subplots(n_var,1, figsize=(10,12))
for j in range(n_var):
    ax = axs.ravel()[j]
    samples[j].plot(axes=ax, group=labels[j], group_colors={f'v{j}': colors[j]}, alpha=0.2, lw=0.95)
    ax.set_title(f'Sampled time series for v{j}')

In [ ]:
# plot basis functions
fig, axs = plt.subplots(n_var, 1, figsize=(10,10))
for j in range(n_var):
    if style is None:
        this_ds = samples[j]
    else:
        basis = get_basis(style, q)
        this_ds = samples[j].to_basis(basis)
        
    # do FPCA
    fpca,_ = get_fpca(n_components=q, style=style)
    fpca.fit(this_ds)
    
    ax = axs[j]
    if style is None:
        W = fpca.components_.data_matrix.squeeze()
        x = np.linspace(time_range[0], time_range[1], W.shape[1])   
        ax.plot(x, W.T, lw=2)       
    else:
        fpca.components_basis.plot(axes=ax)
    #ax.set_ylim(-0.6,0.6)

In [ ]:
sns.heatmap(fpca.components_.coefficients, cmap="coolwarm")

## Reconstruction error

In [ ]:
# plot error for increasing numper of FPCA components
fig, ax = plt.subplots()
    
for j in range(n_var):
    all_err = list()
    
    if style != 'fourier':
        all_q = range(4,10)
    else:
        all_q = [5,7,9,11] # Fourier has always odd number of basis elements
        
    for k in range(len(all_q)):
        if style is None:
            this_ds = samples[j]
        else:
            basis = get_basis(style, all_q[k])
            this_ds = samples[j].to_basis(basis)
    
        fpca,_ = get_fpca(n_components=all_q[k], style=style)
        fpca.fit(this_ds)

        #print("Data shape after FPCA transformation: ", fpca.transform(this_ds).shape)
        
        # reconstruct ds from FPCA
        recov_ds = fpca.inverse_transform(fpca.transform(this_ds))
        
        # reconvert to grid if necessary
        if style is not None:
            recov_ds = recov_ds.to_grid(samples[j].grid_points)
        
        # compute error
        this_err = l2_distance(samples[j], recov_ds) / l2_norm(samples[j])
        all_err.append(np.median(this_err))

   
    ax.plot(all_q, all_err, c=colors[j], lw = 4, marker='p', markersize=9, markeredgecolor='k', alpha=0.6, label=f'v{j}')
    
ax.set_xlabel('Number of FPCA components')
ax.set_ylabel('Median l2 error')
ax.grid(ls = '-', lw = .5) 
ax.legend()

## Functional Graphical Lasso

Now, do for each variable a FPCA with $n_{comp}$ components. Then, concatenate the FPCA coefficients for each sample and compute correlations. With this, we are ready to compute Functional Graphical Lasso.

If we choose $n_{comp}=9$, we expect $4\cdot9=36$ variables in total.

In [ ]:
from gglasso.helper.basic_linalg import scale_array_by_diagonal
from gglasso.helper.utils import lambda_max_fsgl, frob_norm_per_block
from gglasso.solver.functional_sgl_admm import ADMM_FSGL

all_traf = list()
    
# compute FPCA coefficients for each variable
for j in range(n_var):
    if style is None:
        this_ds = samples[j]
    else:
        basis = get_basis(style, n_comp)
        this_ds = samples[j].to_basis(basis)
    
    fpca,_ = get_fpca(n_components=n_comp, style=style)
    fpca.fit(this_ds)

    _traf = fpca.transform(this_ds)
    all_traf.append(_traf)
    
fpca_samples = np.hstack(all_traf)
print("(N,p) = ", fpca_samples.shape)

# compute covariances
S = np.cov(fpca_samples.T)
# scale to correlations
#S = scale_array_by_diagonal(S)

print("S has shape ", S.shape)


In [ ]:
# plot samples
fig, ax = plt.subplots()
sns.heatmap(fpca_samples, ax=ax, alpha=1, vmin=-10, vmax=10, cmap='coolwarm',cbar=True)
ax.vlines([(j+1)*n_comp for j in range(n_var)], 0, n_samples, color='k', lw=4)

# plot heatmap
fig, ax = plt.subplots()
_v = 1.
sns.heatmap(S, ax=ax, cmap="coolwarm", vmax=_v, vmin=-_v, alpha=1.)
ax.hlines([(j+1)*n_comp for j in range(n_var)], 0, n_var*n_comp, color='k', lw=3)
ax.vlines([(j+1)*n_comp for j in range(n_var)], 0, n_var*n_comp, color='k', lw=3)

# plot Frobenius norm of each subblock
fig, ax = plt.subplots()
sns.heatmap(np.round(frob_norm_per_block(S, n_comp),3), annot=True, ax = ax)

In [ ]:
p = n_var
M = n_comp
pM = p*M

lambda_max = lambda_max_fsgl(S, M)
lambda_min = 0.1 * lambda_max

lambda_range = np.logspace(np.log10(lambda_min), np.log10(lambda_max), 10)[::-1]

In [ ]:
lambda_range

In [ ]:
from matplotlib.colors import LogNorm, Normalize

def plot_fsgl_heatmap(sol, p, M, ax=None):
    
    pM = p*M
    
    if ax is None:
        fig, ax = plt.subplots()
    
    _v = 1.
    
    sns.heatmap(scale_array_by_diagonal(sol['Theta']), cmap="coolwarm", vmin=-_v, vmax=_v, ax = ax)
        
    ax.hlines([(j+1)*M for j in range(p)], 0, pM, color='k', lw=3)
    ax.vlines([(j+1)*M for j in range(p)], 0, pM, color='k', lw=3)

    return 

In [ ]:
Omega_0 = np.eye(pM)
all_sol = dict()

for j in range(len(lambda_range)):
    _lam = lambda_range[j]
    
    sol, info = ADMM_FSGL(S, _lam, M, Omega_0,
                         rho=1., max_iter=1000, tol=1e-7, rtol=1e-7,\
                         update_rho=True, verbose=True, measure=True, latent=False, mu1=None)
    
    Omega_0 = sol['Omega']
    all_sol[_lam] = sol.copy()

In [ ]:
fig, axs = plt.subplots(len(lambda_range), figsize=(7,50))

fig2, axs2 = plt.subplots(len(lambda_range), figsize=(5,25))

for j in range(len(lambda_range)):
    _lam = lambda_range[j]

    ax = axs[j]
    plot_fsgl_heatmap(all_sol[_lam], p, M, ax=ax)
    ax.set_title(f"Functional Graphical Lasso for lambda={_lam}")
    
    ax = axs2[j]
    sns.heatmap(frob_norm_per_block(all_sol[_lam]['Theta'], M), vmin=0, vmax=.1, ax = ax)